In [42]:
from mmdet.apis import init_detector, inference_detector, show_result_pyplot
import mmcv

# Specify the path to model config and checkpoint file
config_file = '/home/aiarhipov/centernet/exps/19_eval/config.py'
checkpoint_file = '/home/aiarhipov/centernet/exps/19_eval/epoch60_resnet34_batch2.pth'

# build the model from a config file and a checkpoint file
model = init_detector(config_file, checkpoint_file, device='cuda:0')

# test a single image and show the results
img_path = '/home/aiarhipov/centernet/imgs/0c38706d33642e2c68389688009a4c3958601299.jpg'  # or img = mmcv.imread(img), which will only load it once
res = inference_detector(model, img_path)
result = res
# visualize the results in a new window
# model.show_result(img, result)
# or save the visualization results to image files
# model.show_result(img, result, out_file='result.jpg')
show_result_pyplot(model, img_path, res, score_thr = 0.89, out_file = "/home/aiarhipov/centernet/imgs/output.jpg")

2023-01-13 22:53:22,652 - root - INFO - ModulatedDeformConvPack neck.deconv_layers.0.conv is upgraded to version 2.
2023-01-13 22:53:22,654 - root - INFO - ModulatedDeformConvPack neck.deconv_layers.2.conv is upgraded to version 2.
2023-01-13 22:53:22,656 - root - INFO - ModulatedDeformConvPack neck.deconv_layers.4.conv is upgraded to version 2.


load checkpoint from local path: /home/aiarhipov/centernet/exps/19_eval/epoch60_resnet34_batch2.pth


In [44]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import MinMaxScaler

boxes = result[0]
threshold = 0.7
confident_boxes = boxes[boxes[:, 4] > threshold]
count_confident_boxes = confident_boxes.shape[0]

X = confident_boxes[:, [0, 2]].flatten()
Y = confident_boxes[:, [1, 3]].flatten()

# plt.hist(X, bins=100, label="x")
# plt.hist(Y, bins=100, label="y")

X_scaled = MinMaxScaler().fit_transform(X.reshape((-1, 1)))
Y_scaled = MinMaxScaler().fit_transform(Y.reshape((-1, 1)))

min_samples = int(np.sqrt(count_confident_boxes) / 2)
print("min_samples = ", min_samples)
clustering_X = DBSCAN(eps=0.01, min_samples=min_samples).fit_predict(X_scaled) # .reshape((-1, 1))
clustering_Y = DBSCAN(eps=0.01, min_samples=min_samples).fit_predict(Y_scaled) # .reshape((-1, 1))

labels_X = set(clustering_X)
labels_Y = set(clustering_Y)


img = Image.open(img_path)
draw = ImageDraw.Draw(img)
colors = [(255,0,0), (255,255,0), (255,0,255)]

for x in labels_X:
    x_mean = int(np.mean(X[np.array(clustering_X) == x]))
    print(f"x_label = {x}, x_mean = {x_mean}")
    
    endpoints = (x_mean, 5), (x_mean, 760)
    draw.line(endpoints, fill=colors[0], width=1)
    
for y in labels_Y:
    y_mean = int(np.mean(Y[np.array(clustering_Y) == y]))
    print(f"y_label = {y}, y_mean = {y_mean}")
    
    endpoints = (5, y_mean), (1024, y_mean)
    draw.line(endpoints, fill=colors[0], width=1)
    
img.save("/home/aiarhipov/centernet/imgs/output_lines.jpg")

min_samples =  4
x_label = 0, x_mean = 95
x_label = 1, x_mean = 502
x_label = 2, x_mean = 765
x_label = 3, x_mean = 870
x_label = 4, x_mean = 659
x_label = 5, x_mean = 586
x_label = 6, x_mean = 990
x_label = 7, x_mean = 34
x_label = -1, x_mean = 280
y_label = 0, y_mean = 543
y_label = 1, y_mean = 613
y_label = 2, y_mean = 475
y_label = 3, y_mean = 270
y_label = 4, y_mean = 338
y_label = 5, y_mean = 407
y_label = 6, y_mean = 203
y_label = 7, y_mean = 683
y_label = 8, y_mean = 166
y_label = 9, y_mean = 138
y_label = 10, y_mean = 8
y_label = 11, y_mean = 79
